In [1]:
#Import data wrangling library 
import pandas as pd

#Also import a numerical library - we will need it later 
import numpy as np

In [5]:
#This data is at the school district level
#You will need to replace this with your dataset
TX_exempt = pd.read_csv('C:/Data_various/Texas/TX_exemptions_clean.csv')

#Remove Loving county - no exemptions, and no enrollment data because schools are all in Winkler county
TX_exempt = TX_exempt[TX_exempt.County != 'Loving']

#Select a year of choice (here, 2012) and add a 'Year' column
#Also set county names to lower case for merge later
#This is a good practice - some dataset may have them in lower case, other in upper case, and some with only the first letter capitalized.
TX_exempt_2012 = TX_exempt[TX_exempt['School Year'] == '2012-2013']
TX_exempt_2012['Year'] = 2012
TX_exempt_2012['County'] = TX_exempt_2012['County'].str.lower()

#Import enrollment data for Texas counties in 2012
#Data is broken down by sex (not available unsplit!)
TX_enrollment_sex_2012 = pd.read_csv('C:/Data_various/Texas/TX_Enrollment_2012_sex.csv')
#Only select a couple columns of interest - 'County Name' and 'Enrollment'
#Because the data is split by sex, we need to group by 'County Name' and sum up both sexes 
#Resetting the index ensures that 'County Name' is still a column - we will need it to merge
TX_enrollment_2012 = TX_enrollment_sex_2012[['County Name', 'Enrollment by Gender']].groupby('County Name').sum().reset_index()
#Change the names of the two columns to 'County' (so it matches the other dataset) and 'Enrollment' (because we don't have sex anymore)
TX_enrollment_2012.rename(columns = {'County Name' : 'County', 'Enrollment by Gender' : 'Total_Enrolled'}, inplace = True)
#Also set county names to lower case for merge
TX_enrollment_2012['County'] = TX_enrollment_2012['County'].str.lower()
#Final issue is that this dataset has county names as 'XXX county' but the exemptions only has 'XXX'
#This line splits the 'County' names in two at the first space starting from the back and retains the first part - i.e. 'XXX'
TX_enrollment_2012['County'] = TX_enrollment_2012['County'].str.rsplit(' ', 1).str[0]

#Merge these datasets based on county names, to add enrollment to the exemptions dataset 
TX_clean_2012 = pd.merge(TX_exempt_2012, TX_enrollment_2012, how = 'left', on = 'County')

#Calculate PBE_total - it's the product of the proportion of exemptions by the number enrolled
TX_clean_2012['PBE_total'] = np.round((TX_clean_2012['Percent_PBE'] / 100) * TX_clean_2012['Total_Enrolled'])
#Create a Medical exemptions column for consistency
#Set medical exemptions to 'NaN' - there is no data on that in Texas
TX_clean_2012['PME_total'] = np.nan

#Export the clean file for use elsewhere
TX_clean_2012.to_csv('C:/Data_various/Texas/TX_2012.csv', index = False)


C:\Users\romai\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\Users\romai\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
